In [1]:
import numpy as np

from bokeh.io import curdoc, show
from bokeh.models import ColumnDataSource, GeoJSONDataSource, Grid, LinearAxis, MultiLine, Patches, Plot, ColorBar, LinearColorMapper, BoxZoomTool, ResetTool
from bokeh.palettes import brewer
from bokeh.plotting import figure, show

import meshio 
import celeri 
import pandas as pd 
import geopandas as gpd
import json 

# Read output from pickle file
command, estimation, station, segment, block, meshes = celeri.read_run("/Users/jloveles/Documents/python/celeri/runs/0000000016/")

# Source for filled triangles. Dict of length total n_tde
trisource = ColumnDataSource(dict(
        xtri=[np.array((meshes[i].lon1[j], meshes[i].lon2[j], meshes[i].lon3[j])) for i in range(len(meshes)) for j in range(meshes[i].n_tde)],
        ytri=[np.array((meshes[i].lat1[j], meshes[i].lat2[j], meshes[i].lat3[j])) for i in range(len(meshes)) for j in range(meshes[i].n_tde)],
        sstri=list(estimation["tde_strike_slip_rates"]), 
        dstri=list(estimation["tde_dip_slip_rates"])
    ),
)

# Source for triangle edges. Dict of length n_meshes
triedgesource = ColumnDataSource(dict(
        xedge=[np.array(meshes[i].meshio_object.points[meshes[i].ordered_edge_nodes[:, 0], 0]) for i in range(len(meshes))],
        yedge=[np.array(meshes[i].meshio_object.points[meshes[i].ordered_edge_nodes[:, 0], 1]) for i in range(len(meshes))],
))

# Source for block bounding segments. Dict of length n_segments
segsource = ColumnDataSource(dict(
        xseg=[np.array((segment.loc[i, "lon1"], segment.loc[i, "lon2"])) for i in range(len(segment))],
        yseg=[np.array((segment.loc[i, "lat1"], segment.loc[i, "lat2"])) for i in range(len(segment))],
        sscolor=list(estimation["strike_slip_rates"]),
        dscolor=list(estimation["dip_slip_rates"])
    ),
)

slip_color_mapper = LinearColorMapper(palette=brewer["RdBu"][11], low=-100, high=100)

plot = Plot(
    title=None, width=1000, height=600,
    min_border=10, toolbar_location="below", match_aspect=True, tools=[BoxZoomTool(match_aspect=True)])

triglyph = Patches(xs='xtri', ys='ytri', fill_color={'field': 'sstri', 'transform': slip_color_mapper}, line_width=0)
plot.add_glyph(trisource, triglyph)

triedgeglyph = Patches(xs="xedge", ys="yedge", fill_color=None, line_color='black', line_width=1)
plot.add_glyph(triedgesource, triedgeglyph)

segglyph = MultiLine(xs="xseg", ys="yseg", line_color={'field': 'sscolor', 'transform': slip_color_mapper}, line_width=4)
plot.add_glyph(segsource, segglyph)

xaxis = LinearAxis()
plot.add_layout(xaxis, 'below')

yaxis = LinearAxis()
plot.add_layout(yaxis, 'left')

plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

cbar = ColorBar(color_mapper=slip_color_mapper)
plot.add_layout(cbar, place="right")

curdoc().add_root(plot)

show(plot)

## Reading OSM continent polygons

In [ ]:
# Read simplified OSM continents
# This is fast
coast = gpd.read_file('/Users/jloveles/Downloads/simplified-land-polygons-complete-3857/simplified_land_polygons.shp')
# coast_geo = gpd.read_file('/Users/jloveles/Downloads/gshhg-shp-2.3.7/GSHHS_shp/i/GSHHS_i_L1.shp')
# Convert from Mercator to WGS
# This is fast
coast_geo = coast.to_crs({'init': 'epsg:4326'})
# Convert to json 
# This is slow
coast_json = json.loads(coast_geo.to_json())
# Convert to String like object
json_data = json.dumps(coast_json)
# Write to file
with open('osm_continents.json', 'w') as f:
    f.write(json_data)

In [2]:
with open("osm_continents.json", 'r') as f:
    json_data = f.readlines()

json_data = json_data[0]

In [3]:

# Read from json
# with open('osm_continents.json', "r") as f:
#     coast_json = json.load(f)
# json_data = json.dumps(coast_json)

# plot coastlines

# Input GeoJSON source that contains features for plotting
coastsource = GeoJSONDataSource(geojson = json_data)

plot = Plot(
    title=None, width=1000, height=600,
    min_border=10, toolbar_location="below", match_aspect=True, tools=[BoxZoomTool(match_aspect=True)])

coastglyph = Patches(xs='xs', ys='ys', fill_color='lightgray', line_width=0)
plot.add_glyph(coastsource, coastglyph)

# segglyph = MultiLine(xs="xseg", ys="yseg", line_color={'field': 'sscolor', 'transform': slip_color_mapper}, line_width=4)
# plot.add_glyph(segsource, segglyph)

xaxis = LinearAxis()
plot.add_layout(xaxis, 'below')

yaxis = LinearAxis()
plot.add_layout(yaxis, 'left')

plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

curdoc().add_root(plot)

show(plot)